In [ ]:
# Example 3: correlation C/A-code with data
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget

import gpslib_tutorial as gpslib
from scipy.fft import fft, ifft
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

satNo = 24
dopplerFreq = 0

codeSamples = 2048
NCopies = 1
NGPS = NCopies*codeSamples
SAMPLE_RATE = 2.048e6                                         # per second 
SEC_TIME = np.linspace(1,NGPS,NGPS,endpoint=True,
                       dtype=np.complex64)/SAMPLE_RATE        # in s

DATA_FILE = '../data/230914_data_24s.bin'


def readData(dataFile,ns,smpNo):
    i = 0
    with open(dataFile,'rb') as file:
        while i < smpNo:
            byteData = np.fromfile(file,dtype=np.uint16,count=ns)
            im,re = np.divmod(byteData,256)
            data = np.asarray(re+1j*im,dtype=np.complex64)/127.5 - (1+1j)            
            i += 1
    return data

def fftCorrelate(data,cacode): 
    fftData = fft(data)    
    fftCacode = fft(cacode)
    fftCorr = fftData*np.conjugate(fftCacode) 
    corr = ifft(fftCorr)
    return corr

def demodDoppler(data,dopplerFreq):                          
    factor = np.exp(-1j*(2*np.pi*dopplerFreq*SEC_TIME))
    return factor*data

def calcCorr(freq):
    data = demodDoppler(RAWDATA,freq)                    
    corr = fftCorrelate(data,cacodes)
    return abs(corr)


SMP_NO = 2
RAWDATA = readData(DATA_FILE,NGPS,SMP_NO)
cacodes = gpslib.GPSCacodeRep(satNo,codeSamples,NCopies,0)                                        

fig, ax = plt.subplots(figsize=(6,4))
line, = ax.plot(calcCorr(dopplerFreq))
ax.set_xlabel('Delay $\\tau$')
ax.set_ylabel('Correlation')
ax.set_title('Correlation for satellite PRN%02d after demodulation' % (satNo))
ax.set_ylim(0,100)

# adjust the main plot to make room for the sliders
fig.subplots_adjust(bottom=0.25)
fig.canvas.header_visible = False   

# Make a horizontal slider to control the frequency.
axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='Dopplershift [Hz]',
    valmin=-5000,
    valmax=+5000,
    valinit=dopplerFreq,
)

# The function to be called anytime a slider's value changes
def update(val):
    line.set_ydata(calcCorr(freq_slider.val))
    fig.canvas.draw_idle()

# register the update function for slider
freq_slider.on_changed(update)

plt.show()